# AI Social Security System - Complete Testing Notebook

This notebook provides comprehensive testing of the AI Social Security System including:
- API endpoint testing
- Individual OCR processing
- Individual multimodal analysis
- Complete workflow testing
- System verification

**Prerequisites:**
- API server running on http://localhost:8000
- Streamlit dashboard on http://localhost:8005
- Test documents in docs/ folder

## 1. Setup and Imports

In [ ]:
import requests
import json
import sys
import os
from datetime import datetime
import time

# Add the project root to Python path
sys.path.append('.')

# Import our custom modules
from app.document_processing.ocr_service import OCRService
from app.document_processing.multimodal_service import MultimodalService
from app.decision_making.react_reasoning import ReActDecisionEngine

print("🚀 AI Social Security System - Complete Test Suite")
print("=" * 60)
print(f"📅 Test started at: {datetime.now()}")
print(f"🐍 Python version: {sys.version}")
print("=" * 60)

## 2. System Health Check

In [ ]:
# API endpoints
BASE_URL = "http://localhost:8000"
STREAMLIT_URL = "http://localhost:8005"

def check_system_health():
    print("🏥 SYSTEM HEALTH CHECK")
    print("-" * 40)
    
    # Check API server
    try:
        response = requests.get(f"{BASE_URL}/health/")
        if response.status_code == 200:
            health_data = response.json()
            print("✅ API Server: Running")
            print(f"   Status: {health_data.get('status')}")
            print(f"   Database: {health_data.get('services', {}).get('database', {}).get('status')}")
            print(f"   Redis: {health_data.get('services', {}).get('redis', {}).get('status')}")
        else:
            print(f"❌ API Server: Error {response.status_code}")
    except Exception as e:
        print(f"❌ API Server: Not accessible - {e}")
        return False
    
    # Check Streamlit
    try:
        response = requests.get(STREAMLIT_URL, timeout=5)
        if response.status_code == 200:
            print("✅ Streamlit Dashboard: Running")
        else:
            print(f"⚠️ Streamlit Dashboard: Status {response.status_code}")
    except:
        print("⚠️ Streamlit Dashboard: Not accessible (optional)")
    
    # Check test documents
    emirates_id_path = "docs/EmirateIDFront.jpg"
    bank_statement_path = "docs/Bank Muaaz Alfalah Statement.pdf"
    
    if os.path.exists(emirates_id_path):
        print(f"✅ Emirates ID: Found ({os.path.getsize(emirates_id_path)} bytes)")
    else:
        print(f"❌ Emirates ID: Not found at {emirates_id_path}")
        return False
        
    if os.path.exists(bank_statement_path):
        print(f"✅ Bank Statement: Found ({os.path.getsize(bank_statement_path)} bytes)")
    else:
        print(f"❌ Bank Statement: Not found at {bank_statement_path}")
        return False
    
    print("\n🎯 System ready for testing!")
    return True

# Run health check
system_ready = check_system_health()

## 3. API Discovery and Listing

In [ ]:
def discover_apis():
    print("🔍 API DISCOVERY")
    print("-" * 40)
    
    try:
        # Get OpenAPI spec
        response = requests.get(f"{BASE_URL}/openapi.json")
        if response.status_code == 200:
            openapi_spec = response.json()
            paths = openapi_spec.get('paths', {})
            
            print(f"📋 Total API Endpoints: {len(paths)}")
            print("\n📝 Available Endpoints:")
            
            # Group by category
            categories = {
                'auth': [],
                'workflow': [],
                'documents': [],
                'ocr': [],
                'health': [],
                'analysis': [],
                'other': []
            }
            
            for path in sorted(paths.keys()):
                if '/auth/' in path:
                    categories['auth'].append(path)
                elif '/workflow/' in path:
                    categories['workflow'].append(path)
                elif '/documents/' in path or '/document-management/' in path:
                    categories['documents'].append(path)
                elif '/ocr/' in path:
                    categories['ocr'].append(path)
                elif '/health/' in path:
                    categories['health'].append(path)
                elif '/analysis/' in path:
                    categories['analysis'].append(path)
                else:
                    categories['other'].append(path)
            
            for category, endpoints in categories.items():
                if endpoints:
                    print(f"\n🔸 {category.upper()} APIs ({len(endpoints)}):")
                    for endpoint in endpoints:
                        methods = list(paths[endpoint].keys())
                        print(f"   {', '.join(methods).upper()} {endpoint}")
            
            return paths
        else:
            print(f"❌ Failed to get API spec: {response.status_code}")
            return {}
    except Exception as e:
        print(f"❌ Error discovering APIs: {e}")
        return {}

# Discover all APIs
if system_ready:
    api_paths = discover_apis()
else:
    print("⚠️ System not ready, skipping API discovery")

## 4. User Authentication Setup

In [ ]:
def setup_test_user():
    print("👤 USER AUTHENTICATION SETUP")
    print("-" * 40)
    
    # Generate unique user credentials
    timestamp = int(time.time())
    username = f"notebook_test_{timestamp}"
    email = f"notebook_test_{timestamp}@example.com"
    password = "SecurePass123"
    
    print(f"📝 Creating user: {username}")
    
    # User registration
    register_data = {
        "username": username,
        "email": email,
        "password": password,
        "full_name": "Notebook Test User",
        "phone": "+971501234567",
        "emirates_id": "784-1995-1234567-8"
    }
    
    try:
        # Register
        response = requests.post(f"{BASE_URL}/auth/register", json=register_data)
        
        if response.status_code == 200:
            user_data = response.json()
            print(f"✅ Registration successful")
            print(f"   User ID: {user_data.get('id')}")
            print(f"   Username: {user_data.get('username')}")
        else:
            print(f"⚠️ Registration response: {response.status_code}")
            print(f"   Message: {response.json()}")
            # Try to login with existing user
            username = "muaazbinsaeed"  # Use existing user
            
        # Login
        login_data = {
            "username": username,
            "password": password
        }
        
        response = requests.post(f"{BASE_URL}/auth/login", json=login_data)
        
        if response.status_code == 200:
            auth_data = response.json()
            token = auth_data.get('access_token')
            user_info = auth_data.get('user_info', {})
            
            print(f"✅ Login successful")
            print(f"   Token: {token[:50]}...")
            print(f"   User: {user_info.get('full_name')}")
            
            return {
                'token': token,
                'username': username,
                'user_info': user_info,
                'headers': {'Authorization': f'Bearer {token}'}
            }
        else:
            print(f"❌ Login failed: {response.status_code}")
            print(f"   Error: {response.json()}")
            return None
            
    except Exception as e:
        print(f"❌ Authentication error: {e}")
        return None

# Setup authentication
if system_ready:
    auth_info = setup_test_user()
else:
    print("⚠️ System not ready, skipping authentication")
    auth_info = None

## 5. Individual OCR Testing

In [ ]:
def test_individual_ocr():
    print("🔍 INDIVIDUAL OCR TESTING")
    print("=" * 50)
    
    try:
        # Initialize OCR service
        ocr_service = OCRService()
        print("✅ OCR Service initialized")
        
        # Test 1: Emirates ID
        print("\n📄 TEST 1: EMIRATES ID OCR")
        print("-" * 30)
        
        emirates_path = "docs/EmirateIDFront.jpg"
        print(f"📂 Processing: {emirates_path}")
        
        start_time = time.time()
        eid_result = ocr_service.extract_text(emirates_path)
        end_time = time.time()
        
        print(f"✅ Processing completed in {end_time - start_time:.2f} seconds")
        print(f"📊 Confidence: {eid_result.confidence:.3f} ({eid_result.confidence*100:.1f}%)")
        print(f"📏 Text Length: {len(eid_result.extracted_text)} characters")
        print(f"⏱️ OCR Time: {eid_result.processing_time_ms}ms")
        
        print(f"\n📝 Extracted Text:")
        print("─" * 40)
        print(repr(eid_result.extracted_text))
        print("─" * 40)
        
        # Test 2: Bank Statement
        print("\n📄 TEST 2: BANK STATEMENT OCR")
        print("-" * 30)
        
        bank_path = "docs/Bank Muaaz Alfalah Statement.pdf"
        print(f"📂 Processing: {bank_path}")
        
        start_time = time.time()
        bank_result = ocr_service.extract_text(bank_path)
        end_time = time.time()
        
        print(f"✅ Processing completed in {end_time - start_time:.2f} seconds")
        print(f"📊 Confidence: {bank_result.confidence:.3f} ({bank_result.confidence*100:.1f}%)")
        print(f"📏 Text Length: {len(bank_result.extracted_text)} characters")
        print(f"⏱️ OCR Time: {bank_result.processing_time_ms}ms ({bank_result.processing_time_ms/1000:.1f}s)")
        
        print(f"\n📝 Extracted Text (first 800 chars):")
        print("─" * 40)
        print(bank_result.extracted_text[:800])
        print("...")
        print("─" * 40)
        
        # Summary
        print(f"\n🎯 OCR TESTING SUMMARY:")
        print(f"   Emirates ID: {len(eid_result.extracted_text)} chars @ {eid_result.confidence:.1%}")
        print(f"   Bank Statement: {len(bank_result.extracted_text)} chars @ {bank_result.confidence:.1%}")
        print(f"   Total Time: {(eid_result.processing_time_ms + bank_result.processing_time_ms)/1000:.1f}s")
        
        return {
            'emirates_id': eid_result,
            'bank_statement': bank_result
        }
        
    except Exception as e:
        print(f"❌ OCR Testing failed: {e}")
        import traceback
        traceback.print_exc()
        return None

# Run OCR testing
if system_ready:
    ocr_results = test_individual_ocr()
else:
    print("⚠️ System not ready, skipping OCR testing")
    ocr_results = None

## 6. Individual Multimodal Analysis Testing

In [ ]:
def test_individual_multimodal(ocr_results):
    print("🧠 INDIVIDUAL MULTIMODAL ANALYSIS TESTING")
    print("=" * 50)
    
    if not ocr_results:
        print("❌ No OCR results available for multimodal testing")
        return None
    
    try:
        # Initialize multimodal service
        multimodal_service = MultimodalService()
        print("✅ Multimodal Service initialized")
        
        # Test 1: Emirates ID Analysis
        print("\n🆔 TEST 1: EMIRATES ID MULTIMODAL ANALYSIS")
        print("-" * 40)
        
        eid_text = ocr_results['emirates_id'].extracted_text
        print(f"📝 Input text length: {len(eid_text)} characters")
        
        start_time = time.time()
        eid_analysis = multimodal_service.analyze_document(
            text_content=eid_text,
            document_type='emirates_id',
            file_path='docs/EmirateIDFront.jpg'
        )
        end_time = time.time()
        
        print(f"✅ Analysis completed in {end_time - start_time:.3f} seconds")
        print(f"🎯 Success: {eid_analysis.get('success')}")
        print(f"🔧 Method: {eid_analysis.get('processing_method', 'Intelligent Text Extraction')}")
        
        print(f"\n📋 Extracted Data:")
        extracted_data = eid_analysis.get('extracted_data', {})
        for key, value in extracted_data.items():
            print(f"   {key}: {value}")
        
        # Test 2: Bank Statement Analysis
        print("\n💰 TEST 2: BANK STATEMENT MULTIMODAL ANALYSIS")
        print("-" * 40)
        
        bank_text = ocr_results['bank_statement'].extracted_text
        print(f"📝 Input text length: {len(bank_text)} characters")
        
        start_time = time.time()
        bank_analysis = multimodal_service.analyze_document(
            text_content=bank_text,
            document_type='bank_statement',
            file_path='docs/Bank Muaaz Alfalah Statement.pdf'
        )
        end_time = time.time()
        
        print(f"✅ Analysis completed in {end_time - start_time:.3f} seconds")
        print(f"🎯 Success: {bank_analysis.get('success')}")
        print(f"🔧 Method: {bank_analysis.get('processing_method', 'Intelligent Text Extraction')}")
        
        print(f"\n📋 Extracted Data:")
        extracted_data = bank_analysis.get('extracted_data', {})
        for key, value in extracted_data.items():
            print(f"   {key}: {value}")
        
        # Summary
        print(f"\n🎯 MULTIMODAL ANALYSIS SUMMARY:")
        print(f"   Emirates ID: ✅ {eid_analysis.get('success')} - {len(extracted_data)} fields")
        print(f"   Bank Statement: ✅ {bank_analysis.get('success')} - Financial data extracted")
        print(f"   Processing Method: Intelligent regex-based extraction")
        
        return {
            'emirates_id': eid_analysis,
            'bank_statement': bank_analysis
        }
        
    except Exception as e:
        print(f"❌ Multimodal Analysis failed: {e}")
        import traceback
        traceback.print_exc()
        return None

# Run multimodal testing
if system_ready and ocr_results:
    multimodal_results = test_individual_multimodal(ocr_results)
else:
    print("⚠️ Prerequisites not met, skipping multimodal testing")
    multimodal_results = None

## 7. API Workflow Testing

In [ ]:
def test_api_workflow(auth_info):
    print("🔄 API WORKFLOW TESTING")
    print("=" * 50)
    
    if not auth_info:
        print("❌ No authentication info available")
        return None
    
    headers = auth_info['headers']
    results = {}
    
    try:
        # Test 1: Application Creation
        print("\n📋 TEST 1: APPLICATION CREATION")
        print("-" * 30)
        
        app_data = {
            "full_name": "Notebook Test User",
            "emirates_id": "784-1995-1234567-8",
            "phone": "+971501234567",
            "email": "notebook_test@example.com"
        }
        
        response = requests.post(f"{BASE_URL}/workflow/start-application", 
                               json=app_data, headers=headers)
        
        if response.status_code == 200:
            app_result = response.json()
            app_id = app_result.get('application_id')
            print(f"✅ Application created successfully")
            print(f"   Application ID: {app_id}")
            print(f"   Status: {app_result.get('status')}")
            results['application_id'] = app_id
        else:
            error_data = response.json()
            if 'APPLICATION_EXISTS' in str(error_data):
                app_id = error_data.get('detail', {}).get('existing_application_id')
                if app_id:
                    print(f"✅ Using existing application: {app_id}")
                    results['application_id'] = app_id
                else:
                    print(f"⚠️ Application creation response: {response.status_code}")
                    print(f"   Details: {error_data}")
                    return None
            else:
                print(f"❌ Application creation failed: {response.status_code}")
                print(f"   Error: {error_data}")
                return None
        
        # Test 2: Document Upload
        print("\n📤 TEST 2: DOCUMENT UPLOAD")
        print("-" * 30)
        
        # Prepare files
        emirates_path = "docs/EmirateIDFront.jpg"
        bank_path = "docs/Bank Muaaz Alfalah Statement.pdf"
        
        with open(emirates_path, 'rb') as eid_file, open(bank_path, 'rb') as bank_file:
            files = {
                'emirates_id': ('EmirateIDFront.jpg', eid_file, 'image/jpeg'),
                'bank_statement': ('Bank_Statement.pdf', bank_file, 'application/pdf')
            }
            
            response = requests.post(f"{BASE_URL}/workflow/upload-documents/{app_id}",
                                   files=files, headers={'Authorization': headers['Authorization']})
        
        if response.status_code in [200, 202]:
            upload_result = response.json()
            print(f"✅ Documents uploaded successfully")
            print(f"   Document IDs: {upload_result.get('document_ids')}")
            print(f"   Status: {upload_result.get('status')}")
            print(f"   Progress: {upload_result.get('progress')}%")
            results['upload_result'] = upload_result
        else:
            print(f"❌ Document upload failed: {response.status_code}")
            print(f"   Error: {response.json()}")
        
        # Test 3: Status Check
        print("\n📊 TEST 3: STATUS MONITORING")
        print("-" * 30)
        
        response = requests.get(f"{BASE_URL}/workflow/status/{app_id}", headers=headers)
        
        if response.status_code == 200:
            status_result = response.json()
            print(f"✅ Status retrieved successfully")
            print(f"   Current State: {status_result.get('current_state')}")
            print(f"   Progress: {status_result.get('progress')}%")
            print(f"   Processing Time: {status_result.get('processing_time_elapsed')}")
            
            steps = status_result.get('steps', [])
            print(f"\n   📝 Workflow Steps:")
            for step in steps:
                status_icon = "✅" if step.get('status') == 'completed' else "🔄" if step.get('status') == 'in_progress' else "⏳"
                print(f"      {status_icon} {step.get('name')}: {step.get('message')}")
            
            results['status'] = status_result
        else:
            print(f"❌ Status check failed: {response.status_code}")
            print(f"   Error: {response.json()}")
        
        # Test 4: Processing (if not started)
        if status_result.get('current_state') == 'documents_uploaded':
            print("\n⚙️ TEST 4: START PROCESSING")
            print("-" * 30)
            
            response = requests.post(f"{BASE_URL}/workflow/process/{app_id}", headers=headers)
            
            if response.status_code == 200:
                process_result = response.json()
                print(f"✅ Processing started successfully")
                print(f"   Job ID: {process_result.get('processing_job_id')}")
                print(f"   Estimated time: {process_result.get('estimated_completion')}")
                results['processing'] = process_result
            else:
                print(f"⚠️ Processing start response: {response.status_code}")
                print(f"   Message: {response.json()}")
        
        return results
        
    except Exception as e:
        print(f"❌ API Workflow testing failed: {e}")
        import traceback
        traceback.print_exc()
        return None

# Run API workflow testing
if system_ready and auth_info:
    workflow_results = test_api_workflow(auth_info)
else:
    print("⚠️ Prerequisites not met, skipping API workflow testing")
    workflow_results = None

## 8. Complete AI Workflow with Mock Data

In [ ]:
def test_complete_ai_workflow():
    print("🤖 COMPLETE AI WORKFLOW TEST (MOCK DATA)")
    print("=" * 50)
    
    try:
        # Initialize AI services
        ocr_service = OCRService()
        multimodal_service = MultimodalService()
        react_engine = ReActDecisionEngine()
        
        print("✅ All AI services initialized")
        
        # Step 1: Mock OCR
        print("\n📄 STEP 1: MOCK OCR EXTRACTION")
        print("-" * 30)
        
        emirates_ocr = ocr_service._mock_ocr_extraction("/fake/emirates_id.jpg")
        bank_ocr = ocr_service._mock_ocr_extraction("/fake/bank_statement.pdf")
        
        print(f"✅ Mock Emirates ID: {len(emirates_ocr.extracted_text)} chars")
        print(f"✅ Mock Bank Statement: {len(bank_ocr.extracted_text)} chars")
        
        # Step 2: AI Analysis
        print("\n🧠 STEP 2: AI ANALYSIS")
        print("-" * 30)
        
        emirates_analysis = multimodal_service.analyze_document(
            text_content=emirates_ocr.extracted_text,
            document_type="emirates_id",
            file_path="/fake/emirates_id.jpg"
        )
        
        bank_analysis = multimodal_service.analyze_document(
            text_content=bank_ocr.extracted_text,
            document_type="bank_statement",
            file_path="/fake/bank_statement.pdf"
        )
        
        print(f"✅ Emirates ID Analysis: {emirates_analysis.get('success')}")
        print(f"✅ Bank Statement Analysis: {bank_analysis.get('success')}")
        
        # Step 3: Prepare decision data
        print("\n📋 STEP 3: DECISION PREPARATION")
        print("-" * 30)
        
        applicant_data = {
            'full_name': emirates_analysis.get('extracted_data', {}).get('full_name', 'Mock User'),
            'emirates_id': emirates_analysis.get('extracted_data', {}).get('emirates_id_number', '784-1995-1234567-8'),
            'phone': '+971501234567',
            'email': 'mock@example.com',
            'application_id': 'mock_app_123'
        }
        
        extracted_data = {
            'emirates_id': emirates_analysis.get('extracted_data', {}),
            'bank_statement': bank_analysis.get('extracted_data', {})
        }
        
        application_data = {
            "application_id": "mock_app_123",
            "applicant_data": applicant_data,
            "extracted_data": extracted_data
        }
        
        monthly_income = extracted_data['bank_statement'].get('monthly_income', 0)
        account_balance = extracted_data['bank_statement'].get('account_balance', 0)
        
        print(f"✅ Application prepared for: {applicant_data['full_name']}")
        print(f"   Monthly Income: AED {monthly_income:,.2f}")
        print(f"   Account Balance: AED {account_balance:,.2f}")
        
        # Step 4: ReAct Decision
        print("\n🎯 STEP 4: REACT AI DECISION")
        print("-" * 30)
        
        start_time = time.time()
        decision_result, reasoning_trace = react_engine.make_eligibility_decision(application_data)
        end_time = time.time()
        
        print(f"✅ Decision completed in {end_time - start_time:.3f} seconds")
        
        print(f"\n🎯 DECISION RESULT:")
        print(f"   Outcome: {decision_result['outcome']}")
        print(f"   Confidence: {decision_result['confidence']:.3f}")
        print(f"   Benefit Amount: AED {decision_result['benefit_amount']:,.2f}")
        print(f"   Risk Level: {decision_result.get('risk_level', 'Unknown')}")
        
        reasoning = decision_result.get('reasoning', {})
        print(f"\n📝 REASONING:")
        print(f"   Summary: {reasoning.get('summary', 'N/A')}")
        print(f"   Income Analysis: {reasoning.get('income_analysis', 'N/A')}")
        print(f"   Document Verification: {reasoning.get('document_verification', 'N/A')}")
        
        print(f"\n🔄 REASONING TRACE:")
        print(f"   Steps: {reasoning_trace.total_steps}")
        print(f"   Processing Time: {reasoning_trace.processing_time_ms}ms")
        print(f"   Model: {reasoning_trace.model_used}")
        
        print(f"\n📋 STEP DETAILS:")
        for i, step in enumerate(reasoning_trace.reasoning_steps, 1):
            print(f"   {i}. [{step.step_type.upper()}] {step.content}")
        
        # Final summary
        workflow_summary = {
            "applicant_name": applicant_data['full_name'],
            "monthly_income": monthly_income,
            "final_decision": decision_result['outcome'],
            "confidence_score": decision_result['confidence'],
            "benefit_amount": decision_result['benefit_amount'],
            "processing_time": f"{end_time - start_time:.3f}s",
            "reasoning_steps": reasoning_trace.total_steps
        }
        
        print(f"\n🎉 WORKFLOW COMPLETED SUCCESSFULLY!")
        return workflow_summary
        
    except Exception as e:
        print(f"❌ AI Workflow failed: {e}")
        import traceback
        traceback.print_exc()
        return None

# Run complete AI workflow
if system_ready:
    ai_workflow_results = test_complete_ai_workflow()
else:
    print("⚠️ System not ready, skipping AI workflow testing")
    ai_workflow_results = None

## 9. Quick API Health Tests

In [ ]:
def test_key_apis():
    print("⚡ QUICK API HEALTH TESTS")
    print("=" * 40)
    
    test_results = []
    
    # Define key APIs to test
    api_tests = [
        ("GET", "/health/", "Basic Health", None),
        ("GET", "/health/database", "Database Health", None),
        ("GET", "/", "Root Endpoint", None),
        ("GET", "/openapi.json", "API Spec", None),
    ]
    
    for method, endpoint, description, headers in api_tests:
        try:
            if method == "GET":
                response = requests.get(f"{BASE_URL}{endpoint}", 
                                      headers=headers, timeout=10)
            else:
                response = requests.post(f"{BASE_URL}{endpoint}", 
                                       headers=headers, timeout=10)
            
            if response.status_code < 400:
                print(f"✅ {description}: {response.status_code}")
                test_results.append((description, "PASS", response.status_code))
            else:
                print(f"⚠️ {description}: {response.status_code}")
                test_results.append((description, "WARN", response.status_code))
                
        except Exception as e:
            print(f"❌ {description}: ERROR - {e}")
            test_results.append((description, "FAIL", str(e)))
    
    # Summary
    passed = len([r for r in test_results if r[1] == "PASS"])
    total = len(test_results)
    
    print(f"\n📊 API Health Summary: {passed}/{total} tests passed")
    return test_results

# Run API health tests
if system_ready:
    api_health_results = test_key_apis()
else:
    print("⚠️ System not ready, skipping API health tests")
    api_health_results = []

## 10. Final Test Summary and Report

In [ ]:
def generate_final_report():
    print("📊 FINAL TEST SUMMARY REPORT")
    print("=" * 60)
    print(f"📅 Test completed at: {datetime.now()}")
    print("=" * 60)
    
    # System Status
    print("\n🏥 SYSTEM STATUS:")
    print(f"   API Server: {'✅ Running' if system_ready else '❌ Not available'}")
    if system_ready:
        try:
            response = requests.get(f"{BASE_URL}/health/")
            health_data = response.json()
            services = health_data.get('services', {})
            print(f"   Database: ✅ {services.get('database', {}).get('status', 'Unknown')}")
            print(f"   Redis: ✅ {services.get('redis', {}).get('status', 'Unknown')}")
            print(f"   Models: ✅ {services.get('ollama', {}).get('total_models', 0)} available")
        except:
            print("   Database: ⚠️ Status unknown")
            print("   Redis: ⚠️ Status unknown")
    
    # Test Results Summary
    print("\n📋 TEST RESULTS SUMMARY:")
    
    # OCR Testing
    if ocr_results:
        eid_conf = ocr_results['emirates_id'].confidence
        bank_conf = ocr_results['bank_statement'].confidence
        print(f"   🔍 OCR Testing: ✅ PASSED")
        print(f"      Emirates ID: {eid_conf:.1%} confidence, {len(ocr_results['emirates_id'].extracted_text)} chars")
        print(f"      Bank Statement: {bank_conf:.1%} confidence, {len(ocr_results['bank_statement'].extracted_text)} chars")
    else:
        print(f"   🔍 OCR Testing: ❌ FAILED or SKIPPED")
    
    # Multimodal Testing
    if multimodal_results:
        eid_success = multimodal_results['emirates_id'].get('success', False)
        bank_success = multimodal_results['bank_statement'].get('success', False)
        print(f"   🧠 Multimodal Analysis: ✅ PASSED")
        print(f"      Emirates ID: {'✅' if eid_success else '❌'} {eid_success}")
        print(f"      Bank Statement: {'✅' if bank_success else '❌'} {bank_success}")
    else:
        print(f"   🧠 Multimodal Analysis: ❌ FAILED or SKIPPED")
    
    # API Workflow
    if workflow_results:
        app_id = workflow_results.get('application_id')
        status = workflow_results.get('status', {})
        print(f"   🔄 API Workflow: ✅ PASSED")
        print(f"      Application ID: {app_id}")
        print(f"      Current State: {status.get('current_state', 'Unknown')}")
        print(f"      Progress: {status.get('progress', 0)}%")
    else:
        print(f"   🔄 API Workflow: ❌ FAILED or SKIPPED")
    
    # AI Workflow
    if ai_workflow_results:
        decision = ai_workflow_results.get('final_decision')
        confidence = ai_workflow_results.get('confidence_score')
        print(f"   🤖 AI Workflow: ✅ PASSED")
        print(f"      Decision: {decision}")
        print(f"      Confidence: {confidence:.3f}")
        print(f"      Processing: {ai_workflow_results.get('processing_time')}")
    else:
        print(f"   🤖 AI Workflow: ❌ FAILED or SKIPPED")
    
    # API Health
    if api_health_results:
        passed = len([r for r in api_health_results if r[1] == "PASS"])
        total = len(api_health_results)
        print(f"   ⚡ API Health: {passed}/{total} tests passed")
    
    # Overall Status
    print("\n🎯 OVERALL SYSTEM STATUS:")
    
    components_working = [
        system_ready,
        ocr_results is not None,
        multimodal_results is not None,
        auth_info is not None,
        workflow_results is not None
    ]
    
    working_count = sum(components_working)
    total_components = len(components_working)
    
    if working_count >= 4:
        print(f"   🎉 EXCELLENT: {working_count}/{total_components} components working")
        print(f"   ✅ System is fully operational and production-ready!")
    elif working_count >= 3:
        print(f"   ✅ GOOD: {working_count}/{total_components} components working")
        print(f"   🔧 Minor issues may need attention")
    elif working_count >= 2:
        print(f"   ⚠️ PARTIAL: {working_count}/{total_components} components working")
        print(f"   🛠️ Some components need troubleshooting")
    else:
        print(f"   ❌ ISSUES: {working_count}/{total_components} components working")
        print(f"   🚨 System needs significant attention")
    
    # Quick Access URLs
    print("\n🔗 QUICK ACCESS URLS:")
    print(f"   📚 API Documentation: {BASE_URL}/docs")
    print(f"   📊 Streamlit Dashboard: {STREAMLIT_URL}")
    print(f"   🏥 Health Check: {BASE_URL}/health/")
    print(f"   📋 API Spec: {BASE_URL}/openapi.json")
    
    print("\n" + "=" * 60)
    print("🏁 TESTING COMPLETED")
    print("=" * 60)

# Generate final report
generate_final_report()

## 11. Additional Manual Testing Commands

Run these cells individually for specific testing:

In [ ]:
# Manual API Testing - Modify as needed
def test_specific_api(endpoint, method="GET", data=None, use_auth=True):
    """Test a specific API endpoint"""
    headers = auth_info['headers'] if (use_auth and auth_info) else {}
    
    try:
        if method.upper() == "GET":
            response = requests.get(f"{BASE_URL}{endpoint}", headers=headers)
        elif method.upper() == "POST":
            response = requests.post(f"{BASE_URL}{endpoint}", json=data, headers=headers)
        else:
            print(f"Unsupported method: {method}")
            return
        
        print(f"{method.upper()} {endpoint}")
        print(f"Status: {response.status_code}")
        print(f"Response:")
        print(json.dumps(response.json(), indent=2))
        
    except Exception as e:
        print(f"Error testing {endpoint}: {e}")

# Example usage:
# test_specific_api("/health/database")
# test_specific_api("/auth/me")
print("Manual API testing function loaded. Use test_specific_api() to test individual endpoints.")

In [ ]:
# Test specific OCR on custom file
def test_custom_ocr(file_path):
    """Test OCR on a custom file"""
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        return
    
    try:
        ocr_service = OCRService()
        result = ocr_service.extract_text(file_path)
        
        print(f"OCR Results for: {file_path}")
        print(f"Confidence: {result.confidence:.3f}")
        print(f"Text Length: {len(result.extracted_text)}")
        print(f"Processing Time: {result.processing_time_ms}ms")
        print(f"\nExtracted Text:")
        print("-" * 40)
        print(result.extracted_text[:500] + "..." if len(result.extracted_text) > 500 else result.extracted_text)
        print("-" * 40)
        
        return result
        
    except Exception as e:
        print(f"OCR Error: {e}")
        return None

# Example usage:
# test_custom_ocr("path/to/your/document.pdf")
print("Custom OCR testing function loaded. Use test_custom_ocr('file_path') to test.")

## 12. System Information

View current system configuration and status:

In [ ]:
# Display system information
import platform
import subprocess

print("💻 SYSTEM INFORMATION")
print("=" * 40)
print(f"Platform: {platform.platform()}")
print(f"Python: {platform.python_version()}")
print(f"Architecture: {platform.architecture()}")

# Check if key dependencies are available
print("\n📦 KEY DEPENDENCIES:")
dependencies = [
    'requests', 'fastapi', 'uvicorn', 'sqlalchemy', 
    'pytesseract', 'cv2', 'PIL', 'pandas', 'numpy'
]

for dep in dependencies:
    try:
        if dep == 'cv2':
            import cv2
            print(f"✅ opencv-python: {cv2.__version__}")
        elif dep == 'PIL':
            from PIL import Image
            print(f"✅ Pillow: Available")
        else:
            module = __import__(dep)
            version = getattr(module, '__version__', 'Unknown')
            print(f"✅ {dep}: {version}")
    except ImportError:
        print(f"❌ {dep}: Not installed")

# Check system tools
print("\n🛠️ SYSTEM TOOLS:")
tools = ['tesseract', 'pdftoppm', 'git']

for tool in tools:
    try:
        result = subprocess.run([tool, '--version'], 
                              capture_output=True, text=True, timeout=5)
        if result.returncode == 0:
            version_line = result.stdout.split('\n')[0]
            print(f"✅ {tool}: {version_line}")
        else:
            print(f"⚠️ {tool}: Available but version check failed")
    except (subprocess.TimeoutExpired, FileNotFoundError):
        print(f"❌ {tool}: Not found")

print("\n" + "=" * 40)